In [19]:
import pandas as pd
from scipy import stats
import scikit_posthocs as sp


In [20]:
# --- 1. Load Data ---
# Load all three files into pandas DataFrames.
# Make sure the .csv files are in the same directory as your script.
try:
    df_mft = pd.read_csv("MFT_data.csv")
    df_lus = pd.read_csv("LUS_data.csv")
    df_uhw = pd.read_csv("uhw_data.csv")
except FileNotFoundError:
    print("Error: Make sure all three CSV files (MFT_data.csv, LUS_data.csv, uhw_data.csv) are in the same folder as your script.")
    exit()

# --- 2. Combine Data ---
# Select only the 'Hospital' and 'Score' columns and combine them.
df_combined = pd.concat([
    df_mft[['Hospital', 'Score']],
    df_lus[['Hospital', 'Score']],
    df_uhw[['Hospital', 'Score']]
], ignore_index=True)

In [21]:
# --- 3. Clean Data ---
# Print unique values *before* cleaning to see what we're dealing with
print(f"Unique 'Score' values before cleaning: {df_combined['Score'].unique()}")

# Convert 'Score' column to numeric. 
# 'errors=coerce' turns any non-numeric text (like 'NI', 'ni') into NaN
df_combined['Score_numeric'] = pd.to_numeric(df_combined['Score'], errors='coerce')

# Drop all rows where the numeric score is NaN 
# (which now includes missing values and non-numeric text)
df_cleaned = df_combined.dropna(subset=['Score_numeric'])
# put half of 1.5 in 1 and half in 2
# Deterministically assign 1.5 scores: first half to 1, second half to 2
mask_1_5 = df_cleaned['Score_numeric'] == 1.5
indices_1_5 = df_cleaned[mask_1_5].index
half_point = len(indices_1_5) // 2

df_cleaned.loc[indices_1_5[:half_point], 'Score_numeric'] = 1
df_cleaned.loc[indices_1_5[half_point:], 'Score_numeric'] = 2





# Report the valid, cleaned scores
print(f"Unique 'Score' values after cleaning: {df_cleaned['Score_numeric'].unique()}")

# --- 4. Prepare Groups for Test ---
# Get the unique hospital names from the data
hospital_names = df_cleaned['Hospital'].unique()
print(f"\nFound hospital groups: {hospital_names}")


Unique 'Score' values before cleaning: ['1.0' '2.0' '0.0' '3.0' nan 'NI' '1' '0' '2' 'ni' '3' 1.0 2.0 0.0 1.5]
Unique 'Score' values after cleaning: [1. 2. 0. 3.]

Found hospital groups: ['MFT' 'JCUH' 'UHW']


In [22]:
score_groups = []
for hospital in hospital_names:
    group = df_cleaned[df_cleaned['Hospital'] == hospital]['Score_numeric']
    score_groups.append(group)
    print(f"  - Group: {hospital}, Number of Scores: {len(group)}")


  - Group: MFT, Number of Scores: 1628
  - Group: JCUH, Number of Scores: 1539
  - Group: UHW, Number of Scores: 388


In [23]:
# --- 5. Run the Kruskal-Wallis H-Test ---
# The * operator unpacks the list into separate arguments for the function
if len(score_groups) >= 3:
    statistic, pvalue = stats.kruskal(*score_groups)

    print("\n--- Kruskal-Wallis H-Test Results ---")
    print(f"Test Statistic: {statistic:.4f}")
    print(f"P-value: {pvalue}")

    # Interpretation
    alpha = 0.05
    if pvalue < alpha:
        print("\n---> Interpretation: The p-value is less than 0.05.")
        print("     This suggests there IS a statistically significant difference in the LUS score distributions among the hospitals.")
        print("\n     RECOMMENDATION: Run a Dunn's post-hoc test to see which specific pairs of hospitals are different.")
        
        
        print("\n--- Dunn's Post-Hoc Test ---")
        # Use p_adjust='bonferroni' or 'fdr_bh' (Benjamini-Hochberg)
        dunn_results = sp.posthoc_dunn(df_cleaned, val_col='Score_numeric', group_col='Hospital', p_adjust='bonferroni')
        print(dunn_results)
        
    else:
        print(f"\n---> Interpretation: The p-value is greater than or equal to {alpha}.")
        print("     This suggests there is NO statistically significant difference in the LUS score distributions among the hospitals.")

else:
    print(f"\nError: Expected 3+ hospital groups but found {len(score_groups)}. Cannot run the test.")


--- Kruskal-Wallis H-Test Results ---
Test Statistic: 25.2888
P-value: 3.225561337484064e-06

---> Interpretation: The p-value is less than 0.05.
     This suggests there IS a statistically significant difference in the LUS score distributions among the hospitals.

     RECOMMENDATION: Run a Dunn's post-hoc test to see which specific pairs of hospitals are different.

--- Dunn's Post-Hoc Test ---
          JCUH       MFT       UHW
JCUH  1.000000  0.002986  0.000010
MFT   0.002986  1.000000  0.026937
UHW   0.000010  0.026937  1.000000
